In [1]:
import numpy as np
from TimeTagger import *
import matplotlib.pyplot as plt
from IPython.display import clear_output
import serial
import time
import datetime

In [2]:
# Establish the serial connection
ser = serial.Serial('COM8', 9600)  # Replace 'COM3' with your Arduino's port
tagger = createTimeTagger()

print(ser.readline())
print(ser.readline())
print(ser.readline())

b'Demonstration of AccelStepper Library\r\n'
b"Send 'C' for printing the commands.\r\n"
b'Default speed: 400 steps/s, default acceleration: 800 steps/s^2.\r\n'


In [4]:
def hist(t, bin_width, nbins):
    
    global SPAD_CH, TRIG_CH, tagger
    # print('a0) ',time.time())
    bin_width*=1e12
    t*=1e12 #every time data converted to ps
    hist = Histogram(tagger, SPAD_CH, TRIG_CH, bin_width, nbins)
    hist.startFor(t) #60e12 #in ps
    while hist.isRunning():
        continue
    # print('b) ',time.time())    
    arr = np.array(hist.getData())
    return arr # returns x in picosecond

In [5]:

#Directions are given from the perspective facing HOLMARC sign in FSLab
def send_command(command):
    ser.write(command.encode())

def right_step(dist,speed):
    send_command(f'R{dist} {speed}')
    print(ser.readline())

def left_step(dist,speed):
    send_command(f'E{dist} {speed}')
    print(ser.readline())


def x_step(dist, speed):
    dist*=6400
    speed*=6400
    if xdirection == 0:
        left_step(dist,speed)
    elif xdirection == 1:
        right_step(dist,speed)
    else:
        exit()    

def y_step(dist, speed):
    dist*=6400
    speed*=6400
    if ydirection == 0:
        up_step(dist,speed)
    elif ydirection == 1:
        down_step(dist,speed)
    else:
        exit()

def home_set():
    send_command('X')
    print(ser.readline())


def down_step(dist,speed):
    send_command(f'D{dist} {speed}')
    print(ser.readline())


def up_step(dist,speed):
    send_command(f'U{dist} {speed}')
    print(ser.readline())


def home_step():
    send_command('H')  
    print(ser.readline())
    print(ser.readline())
    print(ser.readline())

def getlocation():
    send_command('L')
    t =  int(ser.readline().split()[-1]), int(ser.readline().split()[-1])
    print(t)
    return t

In [47]:
SPAD_CH = 1 # SPAD Channel
TRIG_CH = 3 # Trigger Channel

# introduce delay in trig channel
delay = 1.04e6 # in picosecond
tagger.setInputDelay(3,delay)

bin_width = 34e-12 # in seconds, check once
c = 3*1e10 #cm

nbins = int(50e-9/bin_width) #2 microsecond of time data
rep_rate = 1 #1kHz
laser_power = np.round(40 * rep_rate,3) #mW

avg_noise_count = 400
# important params: t_acq, bin_width, laser_power

In [56]:
delay = 1.1 #delay between two acquisitions in sec
stepper_speed = 0.1 #cm/sec

total_distance = 12 #cm
n_acq = int((total_distance/stepper_speed)/delay) 

#to move 2cm, stepper motor will need 2*6400 steps. at this speed, it will move for 2*64 = 128 seconds lmao
# works ig
xdirection = 1 # 0 for left, 1 for right
ydirection = 0 # 0 for up, 1 for down
#t_acq = npulse/trigfreq #historgram acquisition time
t_acq = 1

In [57]:
home_set()

time_array = np.arange(delay,delay*n_acq + delay,delay)
time_delay = []

start_time = time.time()
# time.time() --> has a ~millisecond resolution
x_step(total_distance,stepper_speed)
for i in range(n_acq):
    time_delay.append(hist(t_acq,bin_width,nbins))
    if i%100==0:
        print(i)
    while time.time()<=start_time + time_array[i]:
        continue
time_delay = np.array(time_delay)
#plt.plot(time_delay*1e-12*c)
#plt.show()
home_step()

b'Current positions are updated to 0.\r\n'
b'Negative direction for X-axis.\r\n'
0
100
b'Homing X and Y axes.\r\n'
b'X-axis is goimg to the home position.\r\n'
b'Y-axis is at the home position.\r\n'


In [58]:
with open(f'Z:/YRF/Experiments/Imaging/6_march/1D/tacq/details.txt', 'w') as f:
    print(f't_acq: {t_acq} seconds', file=f)
    print(f'rep rate = {rep_rate*1e3}kHz', file=f)
    print(f'laser power = {laser_power}mW', file=f)
    print(f'bin width = {bin_width*1e12}ps', file=f)
    print(f'avg noise count = {avg_noise_count} per second', file=f)

In [59]:
np.savetxt(f'Z:/YRF/Experiments/Imaging/6_march/1D/tacq/{int(bin_width*1e12)}ps_{t_acq}s.txt',time_delay)